# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)
        
print(a)

np.linalg.matrix_rank(a)

[[3.         3.         3.         3.         3.         3.        ]
 [3.         1.875      1.36363636 1.07142857 0.88235294 0.75      ]
 [3.         1.36363636 0.88235294 0.65217391 0.51724138 0.42857143]
 [3.         1.07142857 0.65217391 0.46875    0.36585366 0.3       ]
 [3.         0.88235294 0.51724138 0.36585366 0.28301887 0.23076923]
 [3.         0.75       0.42857143 0.3        0.23076923 0.1875    ]]


6

In [3]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [4]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [5]:
a1 = a.copy()
a1[1, 1] = 3
print(a1)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]]


Resulting matix still has full rank, but the naive LU routine breaks down.

In [6]:
np.linalg.matrix_rank(a1)

6

In [7]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/Users/dmitry_nesterov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/Users/dmitry_nesterov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/Users/dmitry_nesterov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/Users/dmitry_nesterov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [8]:
# ... ENTER YOUR CODE HERE ...
def minor(a, N): #determines the value of minor. Leading minors are given by square matrix of i size, where 1<i<=n 
# and its first element coincide with the first element of an initial matrix
    min_matrix = np.zeros((N, N), dtype=float)
    for i in range (N):
        for j in range (N):
            min_matrix[i,j]=a[i,j]
    return np.linalg.det(min_matrix)

def test(a, N): #checks whether all leading minors are non-zero. If there is any, the function returns 'false'
    result = True
    for i in range(N):
        if (minor(a, i) == 0):
            result = False
    return result

if test(a, N):
    print('Matrix a can be LU-decomposed naively')
else: 
    print('Matrix a cannot be LU-decomposed naively')
    
if test(a1, N):
    print('Matrix a1 can be LU-decomposed naively')
else: 
    print('Matrix a1 cannot be LU-decomposed naively')   

Matrix a can be LU-decomposed
Matrix a1 cannot be LU-decomposed


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [50]:
# ... ENTER YOUR CODE HERE ...

def LU_with_pivoting(a):
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    P = np.eye(N) #permutation matrix is made from indenity matrix
    for j in range(N-1):
        n_max = np.argmax(np.abs(u[j,j:])) + j #finding the number of column where there is a maximum element. 
        
        #We have to add j to argument of maximum element, because we check elements only from j to n and they are
        #numerated starting from 0

        if n_max != j: #it is necessary to swap columns only if the maximum element in each is not on the main diagonal
            P_j = np.eye(N)
            P_j[j,j] = 0 #changing permutation matrix
            P_j[n_max,n_max] = 0 
            P_j[j,n_max] = 1
            P_j[n_max,j] = 1
            u = u @ P_j #swaping columns by right multiplying initial matrix and permutation matrix.
            P = P @ P_j 
            
        lam = np.eye(N) #eliminating the j-th column as usual
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
        
    return L, u, P

In [58]:
L, u, P = LU_with_pivoting(a)
print('Matrix L:', "\n", L, "\n")
print('Matrix U:', "\n", u, "\n")
print('Permutation matrix P:', "\n", P, "\n")
print('Initial matrix a:', "\n", a, "\n")
print('a = L @ U @ P^(-1):', "\n", L @ u @ np.linalg.solve(P, np.eye(N)), "\n") #as AP=LU (column pivoting)

Matrix L: 
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.143 1.    0.    0.    0.   ]
 [1.    1.2   1.65  1.    0.    0.   ]
 [1.    1.231 2.09  2.01  1.    0.   ]
 [1.    1.25  2.406 2.875 2.346 1.   ]] 

Matrix U: 
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -2.475e-01 -1.437e-01 -6.259e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  2.421e-02  2.021e-02  1.030e-02]
 [ 0.000e+00  0.000e+00  0.000e+00 -6.939e-18 -6.462e-04 -5.096e-04]
 [ 0.000e+00  0.000e+00  0.000e+00  1.628e-17  2.168e-19  6.730e-06]] 

Permutation matrix P: 
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

Initial matrix a: 
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 

In [59]:
L, u, P = LU_with_pivoting(a1)
print('Matrix L:', "\n", L, "\n")
print('Matrix U:', "\n", u, "\n")
print('Permutation matrix P:', "\n", P, "\n")
print('Initial matrix a:', "\n", a1, "\n")
print('a = L @ U @ P^(-1):', "\n", L @ u @ np.linalg.solve(P, np.eye(N)), "\n") 

Matrix L: 
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.143 1.    0.    0.    0.   ]
 [1.    1.2   1.179 1.    0.    0.   ]
 [1.    1.231 1.294 1.605 1.    0.   ]
 [1.    1.25  1.375 2.007 2.076 1.   ]] 

Matrix U: 
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -2.475e-01 -1.437e-01 -6.259e-02]
 [ 0.000e+00  0.000e+00  2.220e-16 -9.247e-02 -4.755e-02 -1.920e-02]
 [ 0.000e+00  0.000e+00 -3.565e-16  0.000e+00  1.841e-03  1.181e-03]
 [ 0.000e+00  0.000e+00  2.941e-16 -2.776e-17  4.337e-19 -1.233e-05]] 

Permutation matrix P: 
 [[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

Initial matrix a: 
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 